In [17]:
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
import statsmodels.api as sm
from functools import partial
import dask.array as da
from dask import delayed

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), 'Documents', 'PhD'))
import constants
sys.path.append(constants.MODULE_DIR)

import stats

In [140]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43733,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.18 GiB
Comm: tcp://127.0.0.1:42137,Total threads: 4
Dashboard: /proxy/36203/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:45829,


In [ ]:
# Create sample data
latitudes = np.linspace(-90, 90, 1000)  # Sample latitudes
longitudes = np.linspace(-180, 180, 100)  # Sample longitudes
times = pd.date_range("2024-01-01", periods=500, freq="D")  # 100 time steps

# Create random data for the DataArray
data = np.random.rand(len(times), len(latitudes), len(longitudes))

# Create the xarray DataArray
da = xr.DataArray(
    data,
    dims=("time", "latitude", "longitude"),
    coords={
        "time": times,
        "latitude": latitudes,
        "longitude": longitudes
    }
).chunk({'time':-1, 'latitude':250})

da.name = 'tas'

In [122]:
da

<xarray.DataArray 'tas' (time: 500, latitude: 1000, longitude: 100)>
dask.array<xarray-<this-array>, shape=(500, 1000, 100), dtype=float64, chunksize=(500, 250, 100), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2025-05-14
  * latitude   (latitude) float64 -90.0 -89.82 -89.64 ... 89.64 89.82 90.0
  * longitude  (longitude) float64 -180.0 -176.4 -172.7 ... 172.7 176.4 180.0

In [97]:
def polynomial_fit(y, x=None, order=2, deg=None):
    if np.all(np.isnan(y)):  # Check for all nan values
        return y
    # First need to deal with any nan values at the start or the end
    number_nans_at_start = np.where(~np.isnan(y))[0][0]
    number_nans_at_end = np.where(~np.isnan(y[::-1]))[0][0]

    # Remove these nans
    y = y[number_nans_at_start:]  # Remove start nans
    if number_nans_at_end > 0:
        y = y[:-number_nans_at_end]  # If number_nans_at_end is then this removes all values

    # If x is not provided, generate linearly increasing values
    x = np.arange(len(y)) if x is None else x

    # Perform polynomial fit using numpy's polyfit function
    deg = order if order is not None else deg
    deg = 1 if deg is None else deg
    coeff = np.polyfit(x, y, deg=deg)

    # Generate the fitted line using the computed coefficients
    fitted_line = np.polyval(coeff, x)

    # Re-add the nans back in to maintain the length
    fitted_line_lenght_maintained = np.concatenate([[np.nan] * number_nans_at_start, fitted_line, [np.nan] * number_nans_at_end])
    return fitted_line_lenght_maintained

In [123]:
def apply_polynomial_fit_along_axis(arr, axis):
    return np.apply_along_axis(polynomial_fit, axis, arr)

In [143]:
arr = da.data
axis = da.get_axis_num('time')

In [144]:
polyfit_array = arr.map_blocks(
    apply_polynomial_fit_along_axis,
    axis=axis,
    dtype=arr.dtype)

In [146]:
da_polyfit = xr.zeros_like(da) + polyfit_array

In [147]:
poly_ufunc_da = stats.trend_fit(da, method='POLYNOMIAL', order=2)

In [148]:
%%time
da_polyfit.compute()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 381.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 4.07 s, sys: 3.01 s, total: 7.08 s
Wall time: 13.4 s


<xarray.DataArray (time: 500, latitude: 1000, longitude: 100)>
array([[[0.43746508, 0.45711074, 0.49383272, ..., 0.54356576,
         0.52450585, 0.54177785],
        [0.58290352, 0.42655986, 0.52628786, ..., 0.53442894,
         0.53200352, 0.54601039],
        [0.53041134, 0.47057344, 0.50724947, ..., 0.56734214,
         0.52207715, 0.59924966],
        ...,
        [0.45539739, 0.5110075 , 0.54318454, ..., 0.47224371,
         0.49636612, 0.51789303],
        [0.56523817, 0.53139673, 0.50728283, ..., 0.53295218,
         0.49571497, 0.49610405],
        [0.48920861, 0.50811571, 0.46456395, ..., 0.55444489,
         0.53211506, 0.51049792]],

       [[0.43806822, 0.45732595, 0.49384193, ..., 0.54318916,
         0.52449683, 0.54122245],
        [0.58219628, 0.42721949, 0.52616554, ..., 0.5340572 ,
         0.53202831, 0.5457194 ],
        [0.52997958, 0.47092659, 0.50725174, ..., 0.5668975 ,
         0.52172548, 0.59829116],
...
        [0.5027929 , 0.49804664, 0.50236287, ..., 0.52477803,
         0.56518986, 0.49476333],
        [0.54457732, 0.48497576, 0.49067563, ..., 0.48821628,
         0.48691922, 0.42282024],
        [0.4736892 , 0.48397025, 0.48584602, ..., 0.56510294,
         0.50088019, 0.50572946]],

       [[0.47330844, 0.5317392 , 0.52821584, ..., 0.58247278,
         0.49045591, 0.53254656],
        [0.57217332, 0.49313771, 0.4513535 , ..., 0.44875129,
         0.49992302, 0.50430325],
        [0.49305395, 0.49001572, 0.48131936, ..., 0.52215963,
         0.56792061, 0.51508186],
        ...,
        [0.50257547, 0.4979165 , 0.50264453, ..., 0.52479133,
         0.56565441, 0.4947215 ],
        [0.54522518, 0.48515196, 0.49089572, ..., 0.48826665,
         0.4865992 , 0.42231576],
        [0.4736579 , 0.48378059, 0.48549162, ..., 0.56572956,
         0.50095936, 0.50589208]]])
Coordinates:
  * time       (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2025-05-14
  * latitude   (latitude) float64 -90.0 -89.82 -89.64 ... 89.64 89.82 90.0
  * longitude  (longitude) float64 -180.0 -176.4 -172.7 ... 172.7 176.4 180.0

In [150]:
%%time
poly_ufunc_da.compute()

CPU times: user 2.42 s, sys: 2.14 s, total: 4.56 s
Wall time: 8.96 s


<xarray.DataArray 'tas' (latitude: 1000, longitude: 100, time: 500)>
array([[[0.43746508, 0.43806822, 0.43866923, ..., 0.47422526,
         0.47376791, 0.47330844],
        [0.45711074, 0.45732595, 0.45754091, ..., 0.53157115,
         0.53165531, 0.5317392 ],
        [0.49383272, 0.49384193, 0.49385139, ..., 0.52795889,
         0.52808724, 0.52821584],
        ...,
        [0.54356576, 0.54318916, 0.54281438, ..., 0.58140952,
         0.58194024, 0.58247278],
        [0.52450585, 0.52449683, 0.52448759, ..., 0.4907106 ,
         0.49058337, 0.49045591],
        [0.54177785, 0.54122245, 0.54066921, ..., 0.53151192,
         0.53202816, 0.53254656]],

       [[0.58290352, 0.58219628, 0.58149179, ..., 0.5708476 ,
         0.57150908, 0.57217332],
        [0.42655986, 0.42721949, 0.42787702, ..., 0.49392118,
         0.4935305 , 0.49313771],
        [0.52628786, 0.52616554, 0.52604311, ..., 0.45170943,
         0.45153152, 0.4513535 ],
...
        [0.53295218, 0.53272271, 0.5324938 , ..., 0.48816647,
         0.48821628, 0.48826665],
        [0.49571497, 0.49599846, 0.49628074, ..., 0.48723804,
         0.48691922, 0.4865992 ],
        [0.49610405, 0.49631279, 0.49652009, ..., 0.42332329,
         0.42282024, 0.42231576]],

       [[0.48920861, 0.48917758, 0.48914655, ..., 0.47372049,
         0.4736892 , 0.4736579 ],
        [0.50811571, 0.50820784, 0.5082994 , ..., 0.48415935,
         0.48397025, 0.48378059],
        [0.46456395, 0.46500223, 0.46543892, ..., 0.48619883,
         0.48584602, 0.48549162],
        ...,
        [0.55444489, 0.5538635 , 0.55328454, ..., 0.56447875,
         0.56510294, 0.56572956],
        [0.53211506, 0.53191101, 0.53170754, ..., 0.50080158,
         0.50088019, 0.50095936],
        [0.51049792, 0.51031684, 0.51013644, ..., 0.50556752,
         0.50572946, 0.50589208]]])
Coordinates:
  * time       (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2025-05-14
  * latitude   (latitude) float64 -90.0 -89.82 -89.64 ... 89.64 89.82 90.0
  * longitude  (longitude) float64 -180.0 -176.4 -172.7 ... 172.7 176.4 180.0

# ChatGPT

In [167]:
arr = da.random.uniform(size=(100, 10))
arr

dask.array<uniform, shape=(100, 10), dtype=float64, chunksize=(100, 10), chunktype=numpy.ndarray>

In [168]:
# Wrap the function with dask.delayed
delayed_polynomial_fit = delayed(polynomial_fit)

def apply_polynomial_fit_block(block):
    return delayed_polynomial_fit(block)

In [169]:
# Manually create the Dask graph using da.blockwise
result = da.blockwise(
    apply_polynomial_fit_block,
    'ij', arr,
    'ij', dtype=arr.dtype
)

TypeError: Truth of Delayed objects is not supported